In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from nhdp import nestedHierarchicalNeuralTopicModel
from tsgntm import TreeStructuredGaussianNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import get_hierarchical_affinity, get_topic_specialization, print_topic_sample, print_topic_year
from configure import get_config

pd.set_option('display.max_columns', 30)

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug(variable, sample_batch=None):
    if sample_batch is None: sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch, mode='test')
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

def check(variable):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sample_batch = test_batches[0]
    feed_dict = model.get_feed_dict(sample_batch, mode='test', assertion=True)
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

# model

## initialize model

In [7]:
if 'sess' in globals(): sess.close()
TopicModels = {'hntm': HierarchicalNeuralTopicModel, 'nhdp': nestedHierarchicalNeuralTopicModel, 'tsgntm': TreeStructuredGaussianNeuralTopicModel}
TopicModel = TopicModels[config.model]
model = TopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

# run

## initialize log

In [8]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','','VALID:','','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL', 'GAUSS', 'REG','LOSS','PPL','NLL','KL', 'GAUSS','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))
    
def validate(sess, batches, model):
    losses = []
    ppl_list = []
    prob_topic_list = []
    n_bow_list = []
    n_topics_list = []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, prob_topic_batch, n_bow_batch, n_topics_batch \
            = sess.run([model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, model.prob_topic, model.n_bow, model.n_topics], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
        prob_topic_list.append(prob_topic_batch)
        n_bow_list.append(n_bow_batch)
        n_topics_list.append(n_topics_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    
    probs_topic = np.concatenate(prob_topic_list, 0)
    
    n_bow = np.concatenate(n_bow_list, 0)
    n_topics = np.concatenate(n_topics_list, 0)
    probs_topic_mean = np.sum(n_topics, 0) / np.sum(n_bow)
    
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean, ppl_mean, probs_topic_mean    

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

#         if global_step_log % config.log_period == 0:
        if global_step_log % 1000 == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_gauss_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_gauss_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_gauss_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_gauss_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)
            print(np.sum(debug(model.topic_logvars), 1))
            
            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                                  VALID:        \
       Time  Ep    Ct    LOSS   PPL     NLL    KL GAUSS   REG    LOSS   PPL   
1000     47   0   999  929.74  2714  925.12  1.06  3.31  0.24  892.02  2647   
2000     41   0  1999  923.93  2579  918.36  1.81  3.53  0.23  881.42  2363   
3000     42   1   295  921.08  2498  915.06  2.18  3.62  0.23  879.42  2317   
4000     42   1  1295  918.36  2446  912.01  2.41  3.72  0.22  874.82  2215   
5000     43   1  2295  916.05  2406  909.46  2.58  3.79  0.21  873.10  2192   
...     ...  ..   ...     ...   ...     ...   ...   ...   ...     ...   ...   
234000   56  86  1455  896.93  2032  887.84  4.16  4.83  0.14  862.34  1985   
235000   56  86  2455  896.91  2032  887.82  4.16  4.83  0.14  861.69  1973   
236000   57  87   751  896.90  2032  887.82  4.16  4.83  0.14  863.03  1996   
237000   57  87  1751  896.89  2031  887.81  4.16  4.83  0.14  861.93  1979   
238000   57  88    47  896.89  2031  887.80  4.16  4.83  0.14  861.96  1978   

                                   TEST:       SPEC:             HIER:        
           NLL    KL GAUSS   REG    LOSS   PPL     1     2     3 CHILD OTHER  
1000    888.27  0.85  2.67  0.22  890.74  2605  0.12  0.11  0.12  0.97  0.97  
2000    874.87  2.77  3.60  0.19  881.28  2383  0.17  0.16  0.19  0.96  0.88  
3000    872.31  3.05  3.93  0.13  879.06  2330  0.20  0.22  0.22  0.96  0.76  
4000    866.97  3.51  4.13  0.20  878.40  2317  0.21  0.18  0.21  0.94  0.78  
5000    865.46  3.49  4.08  0.07  874.01  2236  0.19  0.20  0.25  0.94  0.78  
...        ...   ...   ...   ...     ...   ...   ...   ...   ...   ...   ...  
234000  852.79  4.45  4.95  0.14  861.61  2006  0.19  0.22  0.32  0.81  0.63  
235000  852.17  4.48  4.92  0.13  861.61  2006  0.24  0.27  0.33  0.83  0.65  
236000  853.64  4.46  4.83  0.10  861.61  2006  0.20  0.24  0.33  0.84  0.68  
237000  852.51  4.41  4.94  0.08  861.61  2006  0.20  0.23  0.30  0.78  0.58  
238000  852.45  4.43  4.95  0.13  861.61  2006  0.19  0.27  0.29  0.82  0.63  

[238 rows x 22 columns]

0 R: 1.000 P: 0.044 years check free found helpful asked wonderful wanted full told
   1 R: 0.490 P: 0.033 rice soup flavor beef fried spicy tacos dishes thai meat
     11 R: 0.164 P: 0.164 rice mexican thai chinese tacos soup spicy rolls beef noodles
     12 R: 0.064 P: 0.064 soup meat tacos thai beef rice fried fish items found
     13 R: 0.230 P: 0.230 sandwich bread wings steak wine cooked burgers waitress meat italian
   2 R: 0.136 P: 0.012 cream ice tea chocolate sandwich flavor cake flavors coffee tasted
     21 R: 0.005 P: 0.005 sandwich cream ice coffee chocolate cake flavor free tea options
     22 R: 0.087 P: 0.087 cream ice tea cake chocolate flavors flavor donuts boba milk
     23 R: 0.033 P: 0.033 coffee shop starbucks drive store sandwich latte morning sandwiches line
   3 R: 0.029 P: 0.015 years found free today asked drive house helpful check wanted
     31 R: 0.002 P: 0.002 helpful found years wanted told check asked free today care
     32 R: 0.003 P: 0.003 years car

# analysis

In [21]:
def get_topic_year(sample_batches):
    probs_topics = []
    years = []
    for i, sample_batch in sample_batches:
        probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(sample_batch, mode='test'))
        years_batch = [instance.year for instance in sample_batch]
        probs_topics += [probs_topics_batch]
        years += years_batch
    probs_topics = np.concatenate(probs_topics)
    years = np.array(years)

    topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
    topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
    return topic_year

In [22]:
sample_batches = get_batches(instances_train, config.batch_size)
topic_year = get_topic_year(sample_batches)
print_topic_year(sess, model, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2005 level analysis english systems domain lexical linguistic test process lexicon
   1 Avg Year: 2000 grammar structure rules semantic form representation rule lexical type structures
     11 Avg Year: 2006 verb semantic syntactic verbs event relations discourse argument relation arguments
     12 Avg Year: 2007 tree parsing dependency parser grammar trees node parse rules syntactic
   2 Avg Year: 2008 english languages dictionary errors morphological chinese lexical rules corpora pos
     21 Avg Year: 2009 entity sense relations relation semantic entities wordnet senses lexical patterns
     22 Avg Year: 2010 translation english source alignment phrase target languages sentences systems parallel
   3 Avg Year: 2008 models probability training algorithm probabilities search parameters sequence segmentation gram
     31 Avg Year: 2011 features feature training performance learning classifier classification accuracy test class
     32 Avg Year: 2015 models network vector emb